# Comunicados de Política Monetaria
## Parte 2.

En este notebook crearemos un pandas DataFrame con las decisiones de política monetaria de Banxico y haremos un análisis de la información recabada.

Unicamente pandas y numpy, matplotlib, seaborn Con la información obtenida modelar Naive Bayes, si un documento dado pertenece a la clase 'mantien,sube y baja'

Paso:

*Descargar pdfs

*Limpiar los datos 

*Eliminar de todos los archivos las palabras que contengan 'mantiene, incrementa, disminuye'. 

*Parte los documentos entre un set de entrenamiento (0.8) y uno de prueba (0.2) 

*Entrena el modelo con los documentos de entrenamiento y valida el resultado con una matriz de confusión usando la base de prueba.


In [1]:
import re
import os
import numpy as np
import pandas as pd
from requests_html import HTMLSession
from unidecode import unidecode
from tqdm.notebook import tqdm
from time import sleep
from PyPDF4 import PdfFileReader
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix


In [2]:
# Se obtienen los datos generados por la parte 1 desde el archivo pkl.
comunicados = pd.read_pickle('comunicados-banxico.pkl')
comunicados.index


DatetimeIndex(['2008-02-15', '2008-03-14', '2008-04-18', '2008-05-16',
               '2008-06-20', '2008-07-18', '2008-08-15', '2008-09-19',
               '2008-10-17', '2008-11-28',
               ...
               '2018-12-20', '2019-02-07', '2019-03-28', '2019-05-16',
               '2019-06-27', '2019-08-15', '2019-09-26', '2019-11-14',
               '2019-12-19', '2020-02-13'],
              dtype='datetime64[ns]', name='date', length=106, freq=None)

In [3]:

def download_pdf(serie):
    """
    Función para descargar lop PDFs usando como base el Pandas DataFrame genererado en la parte 1
    """
    for link,date,page in zip(serie.url,serie.index,tqdm(range(len(serie)))):
        Pdf="Comunicado-"+str(date)[:10]+".pdf"
        with HTMLSession() as sess:
            r = sess.get(link)
        if not os.path.exists("descargas"):
            os.mkdir("descargas")
        full_path = os.path.join("descargas", Pdf)
        with open(full_path, "wb") as f:
            f.write(r.content)
        sleep(0.5)
    return

In [4]:
download_pdf(comunicados)

In [5]:

def extract_information(pdf_path):
    """
    Función para estraer el contenido de un PDF
    """
    with open(pdf_path, 'rb') as f:
        content = ""
        pdf = PdfFileReader(f)
        for i in range(0, pdf.getNumPages()):
            content += pdf.getPage(i).extractText() + "\n"
            content = " ".join(content.replace("\n", "").strip().split())
            content = " ".join(word.strip(string.punctuation) for word in content.split() if word.isalpha())
    return content


In [6]:
textos = {}

# Con base en el Pandas DataFrame que se generó en la parte 1, se obtienen los contenidos de
# cada PDF descargado usando la funsión extract_information() y se guardan en un diccionario

for date,text in zip(comunicados.index, comunicados.text):
    archivo=os.path.join("descargas", "Comunicado-"+str(date)[:10]+".pdf")
    info=extract_information(archivo)
    category = re.search("(mantiene|incrementa|disminuye)",text).group(1)
    textos.setdefault(category, [])
    textos[category].append(info)


In [7]:
#{nums: len(textos) for nums,textos in textos.items()}
def limpia_texto (textox):
    """
    Función que quita las stopwords, signos de puntuacion y números entregando una lista de palabras
    """
    all_stopwords = [unidecode(word) for word in stopwords.words('spanish')]
    sw_list = ['mantiene','incrementa','disminuye']
    all_stopwords.extend(sw_list)
    textox = " ".join(word.strip(string.punctuation) for word in textox.split() if word.isalpha())
    text_tokens = word_tokenize(textox.lower())
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    return tokens_without_sw

In [8]:
# Se genera cada uno de los arrieglos por clase de documento de acuerdo a la llave del diccionario que los contiene
K1,K2,K3 = "mantiene","incrementa","disminuye"
D1 = [texto for texto in textos[K1]]
D2 = [texto for texto in textos[K2]]
D3 = [texto for texto in textos[K3]]

In [9]:
# Se generan dos sub arreglos por cada unade las clases con los parámetros requeridos 
D1_train, D1_test = train_test_split(D1, test_size=0.2)
D2_train, D2_test = train_test_split(D2, test_size=0.2)
D3_train, D3_test = train_test_split(D3, test_size=0.2)

In [10]:
# Se limpias el texto dejándolo como arreglo de palabras contenidas para cada una de las clases de textos
D1_train = np.concatenate([limpia_texto(text) for text in D1_train ])
D2_train = np.concatenate([limpia_texto(text) for text in D2_train ])
D3_train = np.concatenate([limpia_texto(text) for text in D3_train ])
D1_train

array(['mayo', 'comunicado', 'prensa', ..., 'ciento', 'hacia', 'finales'],
      dtype='<U19')

In [11]:
# Se genera un Pandas DataFrame con los diccionarios de aprendizaje y la probabilidad de cada palabra en la cada clase de documento
dictionary = pd.DataFrame(index=(set(D1_train) | set(D2_train)| set(D3_train)))
dictionary = (dictionary.join(pd.Series(D1_train, name='mantiene').value_counts(),how='left')
                        .join(pd.Series(D2_train, name='incrementa').value_counts(),how='left')
                        .join(pd.Series(D3_train, name='disminuye').value_counts(),how='left'))
dictionary = dictionary.fillna(0)+1
dictionary = dictionary / dictionary.sum(axis=0)
dictionary.head(20)

,mantiene,incrementa,disminuye
ajustadas,0.000075,0.000282,0.000131
exhiba,0.000075,0.000094,0.000131
mantendrá,0.001611,0.000658,0.001051
terremotos,0.000075,0.000094,0.000131
significante,0.000075,0.000094,0.000131
respecto,0.002960,0.002350,0.001840
observarán,0.000075,0.000094,0.000131
detectado,0.000037,0.000282,0.000131
procesos,0.000075,0.000094,0.000131
sustancialmente,0.000112,0.000094,0.000131


In [12]:
# Se obtienes las probabilidades apriori
total = len(D1_train) + len(D2_train) + len(D3_train)
log_prior_D1 = np.log(len(D1_train)/total)
log_prior_D2 = np.log(len(D2_train)/total)
log_prior_D3 = np.log(len(D3_train)/total)
log_Priors = np.array([log_prior_D1,log_prior_D2,log_prior_D3])
log_Priors

array([-0.42220974, -1.5422443 , -2.03635928])

## Se realiza la comprobación

In [13]:
# Se imprimen los tamaños de las pruebas
print("Test de Mantiene: ", len(D1_test), "\nTest de Incrementa", len(D2_test), "\nTest de Disminuye", len(D3_test))

Test de Mantiene:  15 
Test de Incrementa 4 
Test de Disminuye 4


In [14]:
# Se generan las matrices para la prueba
D_test = D1_test + D2_test + D3_test
test = (['mantiene'] * len(D1_test)) + (['incrementa'] * len(D2_test)) + (['disminuye'] * len(D3_test))
hat_test = []
vocs = set(D1_train) | set(D2_train)| set(D3_train)
log_dic = np.log(dictionary)


In [15]:
for Dtest in D_test:
    f = set([palabra for palabra in Dtest.split() if palabra in vocs])
    res = log_dic.loc[f].sum(axis=0) + log_Priors
    hat_test.append(res.idxmax())

In [16]:
hat_test

['mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'incrementa',
 'mantiene',
 'mantiene',
 'mantiene',
 'incrementa',
 'incrementa',
 'incrementa',
 'incrementa',
 'incrementa',
 'mantiene',
 'disminuye',
 'disminuye',
 'disminuye']

In [17]:
test

['mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'mantiene',
 'incrementa',
 'incrementa',
 'incrementa',
 'incrementa',
 'disminuye',
 'disminuye',
 'disminuye',
 'disminuye']

In [18]:
#Se genera la matriz de confusión
confusion_matrix(test, hat_test)

array([[ 3,  0,  1],
       [ 0,  4,  0],
       [ 0,  2, 13]], dtype=int64)

** Nota: La matriz de confusión, no entiendo por qué, me la entrega invertida, teniendo en la esquina inferior derecha el valor para "mantiene" y en la esquina superior izquierda el valor para disminuye